In [2]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.cm as cm
from matplotlib import gridspec

import numpy as np
import pandas as pd
import os
import shutil

#import glob2

#from numba import jit

colors = [
'black','blue','red','green',
'cyan',  
'orange','firebrick','lawngreen','dodgerblue','crimson','orchid','slateblue',
'darkgreen','darkorange','indianred','darkviolet','deepskyblue','greenyellow',
'peru','cadetblue','forestgreen','slategrey','lightsteelblue','rebeccapurple',
'darkmagenta','yellow','hotpink']

Autosaving every 180 seconds


In [ ]:


# def align_get_shifts_with_ref(wf, ref=None, upsample_factor=5, nshifts=7):

#     ''' Align all waveforms on a single channel
    
#         wf = selected waveform matrix (# spikes, # samples)
#         max_channel: is the last channel provided in wf 
        
#         Returns: superresolution shifts required to align all waveforms
#                  - used downstream for linear interpolation alignment
#     '''
#     # Cat: TODO: Peter's fix to clip/lengthen loaded waveforms to match reference templates    
#     n_data, n_time = wf.shape

#     if ref is None:
#         ref = np.mean(wf, axis=0)
      
#     # convert nshifts from timesamples to  #of times in upsample_factor
#     nshifts = (nshifts*upsample_factor)
#     if nshifts%2==0:
#         nshifts+=1

#     # or loop over every channel and parallelize each channel:
#     #wf_up = []
#     wf_up = upsample_resample(wf, upsample_factor)
#     wlen = wf_up.shape[1]
#     wf_start = nshifts//2
#     wf_end = -nshifts//2
    
#     wf_trunc = wf_up[:,wf_start:wf_end]
#     wlen_trunc = wf_trunc.shape[1]
    
#     # align to last chanenl which is largest amplitude channel appended
#     ref_upsampled = upsample_resample(ref[np.newaxis], upsample_factor)[0]
#     ref_shifted = np.zeros([wf_trunc.shape[1], nshifts])
    
#     for i,s in enumerate(range(-(nshifts//2), (nshifts//2)+1)):
#         ref_shifted[:,i] = ref_upsampled[s + wf_start: s + wf_end]

#     bs_indices = np.matmul(wf_trunc[:,np.newaxis], ref_shifted).squeeze(1).argmax(1)
#     best_shifts = (np.arange(-int((nshifts-1)/2), int((nshifts-1)/2+1)))[bs_indices]

#     return best_shifts/np.float32(upsample_factor)

In [14]:
# 
temps = np.load('/media/cat/4TBSSD/data/retina/2009/templates_0sec.npy')
print (temps.shape)

temps1 = temps[10:20]
print (temps1.shape)

max_chans = temps1.ptp(1).argmax(1)

(2413, 101, 512)
(10, 101, 512)


In [33]:
# upsample shift
from scipy import signal

def upsample_resample(wf, upsample_factor):
    wf = wf.T
    waveform_len, n_spikes = wf.shape
    traces = np.zeros((n_spikes, (waveform_len-1)*upsample_factor+1),'float32')
    for j in range(wf.shape[1]):
        traces[j] = signal.resample(wf[:,j],(waveform_len-1)*upsample_factor+1)
    return traces

temp_local = []
for k in range(temps1.shape[0]):
    temp_local.append(temps1[k,:,max_chans[k]])
    
temp_local = np.array(temp_local)

print (temp_local.shape)
upsample_factor = 5
wf_upsampled = upsample_resample(temp_local, upsample_factor)

print (wf_upsampled.shape)

ax = plt.subplot(1,2,1)
t = np.arange(temp_local.shape[1])
plt.plot(t, temp_local.T,c='black')

tt = np.arange(wf_upsampled.shape[1])/upsample_factor+0.5
plt.plot(tt, wf_upsampled.T)

ax = plt.subplot(1,2,2)
shifts = np.arange(0,5,1)
for shift in shifts:
    plt.plot(t, wf_upsampled[0,shift::upsample_factor])


plt.show()

(10, 101)
(10, 501)


ValueError: x and y must have same first dimension, but have shapes (101,) and (100,)

In [28]:
nshifts = 3
ref = np.mean(temp_local, axis=0)
        
wlen = wf_upsampled.shape[1]
wf_start = nshifts//2
wf_end = -nshifts//2

wf_trunc = wf_upsampled[:,wf_start:wf_end]
wlen_trunc = wf_trunc.shape[1]

# align to last chanenl which is largest amplitude channel appended
ref_upsampled = upsample_resample(ref[np.newaxis], upsample_factor)[0]
ref_shifted = np.zeros([wf_trunc.shape[1], nshifts])

for i,s in enumerate(range(-(nshifts//2), (nshifts//2)+1)):
    ref_shifted[:,i] = ref_upsampled[s + wf_start: s + wf_end]
    
print (ref_shifted.shape)

(498, 3)


In [29]:
plt.plot(ref_shifted)
plt.show()

In [ ]:
def shift_chans(wf, best_shifts):
    # use template feat_channel shifts to interpolate shift of all spikes on all other chans
    # Cat: TODO read this from CNOFIG
    wfs_final= np.zeros(wf.shape, 'float32')
    for k, shift_ in enumerate(best_shifts):
        if int(shift_)==shift_:
            ceil = int(shift_)
            temp = np.roll(wf[k],ceil,axis=0)
        else:
            ceil = int(math.ceil(shift_))
            floor = int(math.floor(shift_))
            temp = np.roll(wf[k],ceil,axis=0)*(shift_-floor)+np.roll(wf[k],floor, axis=0)*(ceil-shift_)
        wfs_final[k] = temp
    
    return wfs_final